In [4]:
%pip install git+https://github.com/huggingface/transformers
%pip list | grep -E 'transformers|tokenizers'

  Cloning https://github.com/huggingface/transformers to c:\users\jayor\appdata\local\temp\pip-req-build-3s2a9ks9
  Resolved https://github.com/huggingface/transformers to commit 656e869a4523f6a0ce90b3aacbb05cc8fb5794bb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\jayor\AppData\Local\Temp\pip-req-build-3s2a9ks9'


Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from transformers import GPT2Tokenizer

c:\Users\jayor\anaconda3\envs\chess_transformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
%%time 
from pathlib import Path
#from transformers import GPT2Tokenizer
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("../data/move_files/").glob("**/*.txt")]
print(paths)

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

['..\\data\\move_files\\lichess_elite_games.txt']
CPU times: total: 0 ns
Wall time: 4 ms


In [16]:
tokenizer.save_model("../data/tokenizations/test")

['../data/tokenizations/test\\vocab.json',
 '../data/tokenizations/test\\merges.txt']

In [17]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = GPT2Tokenizer(
    "../data/tokenizations/test/vocab.json",
    "../data/tokenizations/test/merges.txt",
)

In [10]:
"""tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)"""

'tokenizer._tokenizer.post_processor = BertProcessing(\n    ("</s>", tokenizer.token_to_id("</s>")),\n    ("<s>", tokenizer.token_to_id("<s>")),\n)\ntokenizer.enable_truncation(max_length=512)'

In [28]:
tokenizer.encode("e4 e4")

[73, 24, 270, 24]

In [30]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [53]:
if tokenizer.mask_token is None:
    tokenizer.add_special_tokens({'mask_token': '[MASK]'})

In [54]:
import torch
torch.cuda.is_available()

True

In [55]:
from transformers import GPT2Config

In [56]:
config = GPT2Config(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [57]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel(config=config)

In [58]:
model.num_parameters()

82859520

Build training dataset

In [59]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="../data/move_files/lichess_elite_games.txt",
    block_size=128,
)

CPU times: total: 0 ns
Wall time: 8 ms


c:\Users\jayor\anaconda3\envs\chess_transformer\lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [60]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

In [61]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="../data/output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [62]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
c:\Users\jayor\anaconda3\envs\chess_transformer\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  3.18it/s]

{'train_runtime': 0.3131, 'train_samples_per_second': 31.942, 'train_steps_per_second': 3.194, 'train_loss': 11.078603744506836, 'epoch': 1.0}
CPU times: total: 141 ms
Wall time: 322 ms


TrainOutput(global_step=1, training_loss=11.078603744506836, metrics={'train_runtime': 0.3131, 'train_samples_per_second': 31.942, 'train_steps_per_second': 3.194, 'train_loss': 11.078603744506836, 'epoch': 1.0})

In [63]:
trainer.save_model("../data/tokenizations/test")

In [66]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="../data/tokenizations/test",
    tokenizer="../data/tokenizations/test"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PipelineException: The tokenizer does not define a `mask_token`.